# HW2 - Pandas and ICD-codes

### Get the data

For this assignment, we'll need to get some data! We will be using the Diabetes Dataset that is located here:

https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008#

Afterwards, unzip the folder and place the contents in a folder called `/data/` at this directory.

Make sure to add a file called `.gitignore` at the root of your directory and add the line 

```
    data/
```
to it so that it ignore any files that you place in the Data folder.

In [1]:
import pandas as pd
import numpy as np
import string
import re

In [252]:
diabetes_df = pd.read_csv('./data/diabetic_data.csv')
adm_source_id_map = pd.read_csv('./data/admissions_source_id_map.csv')
adm_type_id_map = pd.read_csv('./data/admission_type_id_map.csv')
disch_disp_id_map = pd.read_csv('./data/discharge_disposition_id_map.csv')

![](data/datadictionary.png)

## Data Source [5 pts]

When we begin working with data, it is important to try and understand the data that we've been given. Often the context can tell us a lot of information about the data. In fact, often times understanding what is *not* in the data is just as critical as understanding the data itself. From the above link, the information about how the data was collected can be found in this [paper](https://www.hindawi.com/journals/bmri/2014/781670/)

List the 5 inclusion criteria to be in the dataset: 

1. 
2. 
3. 
4. 
5. 


## Explore the Data [55 pts]
Using the data, answer the following questions:

#### 1. How many rows does the `diabetes_df` have? How many columns? [2.5 pts]

#### 2. How many unique encounters are there? How many unique patients? [2.5 pts]

#### 3. What is the most amount of encounters that a single patient has in the dataset? [5 pts]

#### 4. Show the proportion of non-missing values are in the dataset for each column. [10 pts]
> Make sure you check to see if there are missing values that aren't coded as missing, but should be

#### 5. For all numeric columns, show summary statistics (mean, median, max, min, etc) [2.5 pts]

#### 6. For all columns with a `dtype` of object, show the count of all of the values in that column [5 pts]
> You may want to reference the `pd.DataFrame.select_dtypes()` function 

#### 7. What is the average number of labs administered by age category [2.5 pts]

#### 8. Does the number of diagnoses equal the number of non-NA entries in the diag_* columns? [2.5 pts]

#### 9. Create a new column that has the value of 1 if the medical specialty in that row contains the word Surgery and 0 otherwise. [10 pts]

#### 10. How many encounters where the patient was between the ages of 0 and 20 took place with Pediatric providers? [5 pts]

#### 11. Join in the admission source, admission_type_id, and discharge_disposition_id to the diabetes dataframe. Use the default inner join method [5 pts]

#### 12. find the average A1C (if available) for patients that were readmitted vs. not readmitted patients [2.5 pts]


## Single-level CCS categories [40 pts]

The columns `diag_1`, `diag_2`,  and `diag_3` contain ICD-9-CM codes for the encounters that took place in this dataset. However, if we count up the number of unique values between the 3 columns, we can see that the data is very sparse. 

As we discussed in class, the single-level CCS categories can be used instead to group similar ICD codes together. Download the latest version of the ICD-9-CM single-level CCS here: [https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip](https://www.hcup-us.ahrq.gov/toolssoftware/ccs/Single_Level_CCS_2015.zip)

Unzip this and put the contents in the `data` folder along with the Diabetes dataset and read the file called `$dxref 2015.csv` into a variable. **NOTE** You must skip the first row of this `csv` file when reading it in because there is a note there that is unrelated to the contents of the data. Look up how to do this using the `pd.read_csv` function

Examine the contents of the data. Please bear in mind that Jupyter notebooks do not render whitespace or quotes very well sometimes, so watch out for that. Make sure you examine column names with `.columns` instead of just calling `.head()` and visually inspecting, for example.

If you notice, the ICD codes are not very well-formatted in either the Diabetes dataset (for example, the code `8` should really be `008.0`) or the Single-level CCS crosswalk (`' and whitespace characters`). 

This is quite typical of healthcare data, unfortunately. Many of the publically available files are not suited for reading into modern programming languages. Often, they are limited to SAS format datasets, which is a proprietary software suite for working with statistical packages that is ubiquitous in health care.

In order to use the CCS groupings, we'll have to clean both the groupings *and* the diabetes data as well. Here is the general procedure that we will take (Although this is an imperfect mapping as well).

**Remove all quotes and extra whitespace from the codes and the column names in the CCS crosswalk [10 pts]**

Next, we're going to write a function that cleans up the ICD codes found in the Diabetes dataset. If you'll notice, there are no decimal points in the single-level CCS crosswalk. Therefore, we must make sure that our data matches that as well. Implement the function below, and we will use it in an `.apply()` call to modify the `diag_` columns.

#### Implement this function [15 pts]

In [2]:
def clean_diabetes_code(icd_code):
    """
    Formats codes found in the Diabetes dataset to be like those found in the CCS crosswalk
    
    
    If an icd_code has a decimal, remove the decimal
    
    if it has less than 3 digits, prepend '0's, until it is 3 digits.
    
    If it has 3 digits (before or after the above step), append a '0'
    
    Examples:
        250.13 -> 25013
        32 -> 0320
        315 -> 3150
        
    Args: 
        icd_code:
    
    Returns:
        formatted_code string: A formatted ICD-Code string according to the above criteria:
        
        
    """
    code_copy = str(icd_code) # Convert to string due to weird .apply behavior in Series
    if code_copy == 'nan':
        return np.nan
    
    ### Your Code here: -- modify code_copy until it returns the reformatted code. Store this in formatted_code and return it

Now, replace all three `diag_` columns by calling .apply with this function. For example:

`diabetes_df['diag_1'] = diabetes_df['diag_1'].apply(clean_diabetes_code)`

#### Join in the single-level CCS crosswalk and answer the following question: [15 pts]

List the top 10 condition categories (`CCS CATEGORY DESCRIPTION`) when you add up all instances over all 3 columns